<a href="https://colab.research.google.com/github/YaninaK/anomaly-detection/blob/b1/notebooks/01_Missing_consumption_records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обнаружение аномалий в начислениях за тепловую энергию

## 1. Нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель)

### Примеры аномалий

Виды аномалий по показаниям приборов учёта тепловой энергии, которые необходимо выявлять (кроме объектов с видом энергопотребления ГВС (централ):

1. нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель);

2. равные значения показаний в течение нескольких расчетных периодов;

3. снижение/рост показаний в отдельные месяцы по сравнению с показаниями за предыдущие периоды по данному объекту (с учётом фактической температуры наружного воздуха и количества отопительных дней в месяце);

4. аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами (только для типов объекта «Многоквартирный дом») по критериям:
  - год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
  - этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
  - площадь (±10%),
  - наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).



In [1]:
initiate = False
if initiate:
  !git init -q
  !git clone -b b1  https://github.com/YaninaK/anomaly-detection.git -q

  from google.colab import drive
  drive.mount('/content/drive')

  !unzip -u -q /content/drive/MyDrive/ML_projects/08_anomaly_detection/data/01_raw/task#3.zip -d /content/anomaly-detection/data/01_raw

%cd /content/anomaly-detection

/content/anomaly-detection


In [2]:
import os
import sys

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "data"))
sys.path.append(os.path.join(os.getcwd(), "models"))
sys.path.append(os.path.join(os.getcwd(), "src", "anomaly_detection"))

In [3]:
import numpy as np
import pandas as pd

from data.make_dataset import load_data
from features.missing_records import (
    missing_data_and_nonunique_objects_detection_pipeline,
)

In [4]:
import warnings
warnings.filterwarnings('ignore')

## 1. Чтение данных

In [5]:
%%time
save = False
data, temperature, buildings = load_data(save=save)

CPU times: user 1.49 s, sys: 59.7 ms, total: 1.55 s
Wall time: 2.77 s


## 2. Обнаружение нулевых значений показаний за тепловую энергию в отопительный период.

In [6]:
%%time
missing_consumption_records, uninvoiced_objects, nonunique_objects = (
    missing_data_and_nonunique_objects_detection_pipeline(data, buildings, temperature)
)

CPU times: user 4.81 s, sys: 89.2 ms, total: 4.9 s
Wall time: 7.6 s


### 2.1. Пропуски в данных по текущему потреблению

In [7]:
print(f"missing_records.shape = {missing_consumption_records.shape}\n")
missing_consumption_records.sample(2)

missing_records.shape = (1053, 14)



,,,,,2021-10-01 00:00:00,2021-11-01 00:00:00,2021-12-01 00:00:00,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00
Адрес объекта,Тип объекта,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта 2,,,,,,,,,,,,,,
"г Уфа, ул. Владивостокская, д.12, Подобъект №984637",Многоквартирный дом,186248,0,"г Уфа, ул. Владивостокская, д.12",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN
"г Уфа, проезд. Лесной, д.3",Другое строение,21753,1,"г Уфа, проезд. Лесной, д.3",0.0,0.0,2212.7999,2660.4185,2480.57,2084.06,1213.639,NaN,NaN,2591.5,3102.458,2591.021,1770.85,1063.2627


### 2.2 Объекты без данных по текущему потреблению

In [8]:
print(f"uninvoiced_objects.shape = {uninvoiced_objects.shape}")
uninvoiced_objects.sample(2)

uninvoiced_objects.shape = (1587, 6)


,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2
1565,"г Уфа, ул. Юрия Гагарина, д.10 корп.4",Другое строение,2,2014-01-01,0.01,"г Уфа, ул. Юрия Гагарина, д.10 корп.4"
573,"г Уфа, ул. Кольцевая, д.113 корп.а",Многоквартирный дом,2,1952-01-01,416.90,"г Уфа, ул. Кольцевая, д.113 корп.а"


In [9]:
pd.concat(
    [
        uninvoiced_objects["Тип Объекта"].value_counts(),
        uninvoiced_objects["Тип Объекта"].value_counts(normalize=True)
    ], axis=1
).head(10)

,count,proportion
Тип Объекта,,
Многоквартирный дом,752,0.473850
Другое строение,617,0.388784
Частный дом,138,0.086957
"Административные здания, конторы",30,0.018904
"Учебное заведение, комбинат, центр",12,0.007561
Школы и ВУЗ,10,0.006301
"Жилое здание (Гостиница, Общежитие)",5,0.003151
Производственный объект,3,0.001890
Гаражные комплексы,3,0.001890


In [10]:
cond = uninvoiced_objects["Тип Объекта"] == "Многоквартирный дом"
s = uninvoiced_objects[cond]['Общая площадь объекта'].sum() / 1e06

print(f"У {cond.sum()} объектов типа Многоквартирный дом общей площадью{s: 0.1f} млн. нет данных о выствленных счетах.")

У 752 объектов типа Многоквартирный дом общей площадью 1.1 млн. нет данных о выствленных счетах.


* У 1587 объектов нет данных об учете потребления теплоэнергии в осенне-зимний период. Из них 752 объекта (47%) общей площадью 1.1 млн. приходится на тип ```Многоквартирный дом```, около 39% - на тип ```Другое строение```, около 9% - на тип ```Частный дом```.


### 2.3 Неуникальные адреса объектов.

In [11]:
print(f"Всего неуникальных объектов: {nonunique_objects.shape[0]}\n")
nonunique_objects.head(6)

Всего неуникальных объектов: 145



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2
122,"г Уфа, б-р. Тухвата Янаби, д.34",Другое строение,1,NaT,0.01,"г Уфа, б-р. Тухвата Янаби, д.34"
123,"г Уфа, б-р. Тухвата Янаби, д.34",Другое строение,2,NaT,0.01,"г Уфа, б-р. Тухвата Янаби, д.34"
412,"г Уфа, пр-кт Октября, д.153",Другое строение,0,NaT,0.01,"г Уфа, пр-кт Октября, д.153"
413,"г Уфа, пр-кт Октября, д.153",Другое строение,3,NaT,0.01,"г Уфа, пр-кт Октября, д.153"
487,"г Уфа, пр-кт Октября, д.31",Другое строение,0,1970-01-01,0.01,"г Уфа, пр-кт Октября, д.31"
488,"г Уфа, пр-кт Октября, д.31",Другое строение,5,1970-01-01,0.01,"г Уфа, пр-кт Октября, д.31"


In [12]:
pd.concat(
    [
      nonunique_objects["Тип Объекта"].value_counts(),
      nonunique_objects["Тип Объекта"].value_counts(True),
      nonunique_objects.groupby(["Тип Объекта"])["Адрес объекта"].agg(lambda x: x.nunique())
    ], axis=1
)

,count,proportion,Адрес объекта
Тип Объекта,,,
Другое строение,116,0.800000,51
Многоквартирный дом,14,0.096552,7
Частный дом,12,0.082759,4
"Учебное заведение, комбинат, центр",3,0.020690,1


1. Типы объектов ```Другое строение``` и ```Частный дом``` оставлены без корректировок:
  * Уникальный адрес объекта нужен для привязки площади объекта и даты постройки к данным учета потребления теплоэнергии. У типа ```Другое строение``` - в большинстве случаев эти данные отстутствуют, соответственно этот блок информации не имеет смысла корректировать.

  * Информации об неуникальных объектах типа ```Частный дом``` нет в данных о потреблении теплоэнергии - этот блок также не имеет смысла корректировать.

2. Блок ```Многоквартирный дом``` скорректирован:
  * Адреса многоквартирных домов сделаны уникальными в признаке ```Адрес объекта 2``` через сопоставление с данными об учете теплоэнергии.
  * На трех объектах ниже счета не выставляются, для них в признаке ```Адрес объекта 2``` к адресу добавлено слово ```extra``` :
    * ```г Уфа, ул. Вологодская, д.20```,
    * ```г Уфа, ул. Интернациональная, д.113```,
    * ```г Уфа, ул. Нежинская, д.6```.
3. Блок ```Учебное заведение, комбинат, центр``` скорректирован:
  * Адреса сделаны уникальными в признаке ```Адрес объекта 2``` через сопоставление с данными об учете теплоэнергии и проставление ``№ ОДПУ`` в адрес.

In [13]:
nonunique_2 = buildings[
    buildings.duplicated(subset=["Адрес объекта 2", "Тип Объекта"], keep=False)
]
pd.concat(
    [
      nonunique_2["Тип Объекта"].value_counts(),
      nonunique_2["Тип Объекта"].value_counts(True),
      nonunique_2.groupby(["Тип Объекта"])["Адрес объекта 2"].agg(lambda x: x.nunique())
    ], axis=1
)

,count,proportion,Адрес объекта 2
Тип Объекта,,,
Другое строение,116,0.90625,51
Частный дом,12,0.09375,4


* Если использовать ```Адрес объекта 2```, неуникальные адреса в типах  ```Многоквартирный дом``` и ```Учебное заведение, комбинат, центр``` появляться не будут.